In [ ]:
#@markdown ## 구글 드라이브 전체 설치 (최초 한번만 설치)
#@markdown <div><font color="red">5GB~7GB정도 필요해요. 구글드라이브 용량을 확인해주세요</font></div>  
#@markdown * 설치후에 <a href="https://github.com/ninjaneural/webui/blob/master/memo/install.md">이곳</a>에서 실행방법을 확인해주세요*
Google_Install = True

#@markdown **WebUI Version**
TagVersion = 'v1.7.0' #@param ["v1.3.2", "v1.4.1", "v1.5.2", "v1.6.0", "v1.7.0"]

#@markdown **폴더 설명** 
#@markdown * install/outputs : 생성된 이미지들이 저장되요
#@markdown * install/models/Stable-diffusion/ : 모델(checkpoint)를 넣어주면 읽어올수 있어요
#@markdown * install/models/Lora : 로라(LoRA)를 넣어주면 읽어올수 있어요
#@markdown * install/models/LyCORIS : 라이코리스(LyCORIS)를 넣어주면 읽어올수 있어요
#@markdown * install/embeddings : 임베딩(Textual Inversion)를 넣어주면 읽어올수 있어요
#@markdown * install/hypernetworks : 하이퍼워크(Hyperworks)를 넣어주면 읽어올수 있어요

#@markdown **초기모델(checkpoint) URL**
Checkpoint_Url = 'https://civitai.com/api/download/models/149660?type=Model&format=SafeTensor&size=pruned&fp=fp16' #@param {type:"string"}
#@markdown **초기모델 파일명**
Checkpoint_Filename = 'cherry_picker_xl.safetensors' #@param {type:"string"}

#@markdown -----
#@markdown *추가익스텐션*

ControlNet = True 
ADetailer = True

#@markdown **Dectect Detailer(ddetailer)를 사용하지 않으면 체크를 해제해주세요**
DDetailer = False #@param {type:"boolean"}

#@markdown **Segment Anything을 사용하지 않으면 체크를 해제해주세요**
SegmentAnything = False #@param {type:"boolean"}

NotebookVersion = 'install'
Workspace = 'drive/MyDrive/install'

if Google_Install:
  from google.colab import drive
  drive.mount('/content/drive')

if Google_Install:
  !mkdir -p /content/{Workspace}

Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion'

!apt -y install -qq aria2

%cd /content
!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/install_{NotebookVersion}.sh -O /content/install.sh
!bash /content/install.sh {Workspace} {TagVersion} {ControlNet} {DDetailer} {ADetailer} {SegmentAnything}

%cd /content/{Workspace}

# checkpoint select
!sed -i -e 's/"sd_model_checkpoint": "",/"sd_model_checkpoint": "{Checkpoint_Filename}",/g' /content/{Workspace}/config.json

# upscaler
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/{Workspace}/models/ESRGAN -o 4x-UltraSharp.pth

# checkpoint
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d {Checkpoint_SavePath} -o {Checkpoint_Filename}

# vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {Checkpoint_SavePath} -o vae-ft-mse-840000-ema-pruned.vae.pt

# lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/62833?type=Model&format=SafeTensor" -d /content/{Workspace}/models/Lora -o add_detail.safetensors

# embeddings
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d /content/{Workspace}/embeddings -o EasyNegative.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d /content/{Workspace}/embeddings -o badhandv4.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60938?type=Negative&format=Other" -d /content/{Workspace}/embeddings/ -o negative_hand.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60095?type=Negative&format=Other" -d /content/{Workspace}/embeddings/ -o bad_prompt_version2.pt

!git clone https://github.com/salesforce/BLIP.git /content/drive/MyDrive/install/repositories/BLIP
%cd /content/drive/MyDrive/install/repositories/BLIP
!git reset --hard 48211a1594f1321b00f14c9f7a5b4813144b2fb9

!git clone https://github.com/sczhou/CodeFormer.git /content/drive/MyDrive/install/repositories/CodeFormer
%cd /content/drive/MyDrive/install/repositories/CodeFormer
!git reset --hard c5b4593074ba6214284d6acd5f1719b6c5d739af

!git clone https://github.com/Stability-AI/generative-models.git /content/drive/MyDrive/install/repositories/generative-models
%cd /content/drive/MyDrive/install/repositories/generative-models
!git reset --hard 45c443b316737a4ab6e40413d7794a7f5657c19f

!git clone https://github.com/crowsonkb/k-diffusion.git /content/drive/MyDrive/install/repositories/k-diffusion
%cd /content/drive/MyDrive/install/repositories/k-diffusion
!git reset --hard ab527a9a6d347f364e3d185ba6d714e22d80cb3c

!git clone https://github.com/Stability-AI/stablediffusion.git /content/drive/MyDrive/install/repositories/stable-diffusion-stability-ai
%cd /content/drive/MyDrive/install/repositories/stable-diffusion-stability-ai
!git reset --hard cf1d67a6fd5ea1aa600c4df58e5b47da45f6bdbf

print("------------------------------------------------------------------------------------------------")
print("설치완료!")
print("https://github.com/ninjaneural/webui/blob/master/memo/install.md 이 페이지를 참고해서 실행해주세요")
